This notebook is to create OpenMM systems to run hydration free energy (HFE) calculations using GAFF, SMIRNOFF and your choice of water models.

Dependencies:
- openmm
- openff-toolkit
- openmmforcefields 

In [ ]:
from openff.toolkit.topology import Molecule as Molecule
from openff.toolkit.topology import Topology as Topology
from openff.toolkit.typing.engines.smirnoff import ForceField

from simtk import openmm
from simtk.openmm.app import PME, NoCutoff, HBonds, PDBFile
from simtk.unit import *
from openmmforcefields.generators import (
    GAFFTemplateGenerator,
    SMIRNOFFTemplateGenerator,
)

### System information

In [ ]:
system_name = "molecule"
system_smiles = "CCO"
forcefield_name = "SAGE"
temperature = 298 * kelvin

system_ff_filename = f"{system_name}_forcefield_{forcefield_name}.xml"
system_pdb_filename = f"{system_name}.vac.pdb"
solvated_system_pdb_filename = f"{system_name}.sol.pdb"
saved_vacumm_system_file_name = (
    f"{system_name}_vac_system_{forcefield_name.lower()}.xml"
)
saved_solvated_system_file_name = (
    f"{system_name}_sol_system_{forcefield_name.lower()}.xml"
)

### Create system 
Here we create a system from a SMILES string

In [ ]:
off_mol = Molecule.from_smiles("CCO")
off_mol.generate_conformers(n_conformers=1)
off_mol.compute_partial_charges_am1bcc()
off_topology = Topology.from_molecules(off_mol)

omm_vac_topology = off_topology.to_openmm()
omm_vac_position = off_mol.conformers[0]

### Create Openmm system

In [ ]:
forcefield = openmm.app.ForceField("tip3p.xml")

if forcefield_name.upper() == "GAFF":
    template = GAFFTemplateGenerator(molecules=[off_mol])
elif forcefield_name.upper() == "SAGE":
    template = SMIRNOFFTemplateGenerator(
        forcefield="openff-2.0.0.offxml", molecules=[off_mol]
    )

else:
    print("Unknow force field, cannot preceed.")

print("Using force field:", forcefield_name)
forcefield.registerTemplateGenerator(template.generator)

In [ ]:
modeller = openmm.app.Modeller(omm_vac_topology, omm_vac_position)

modeller.addSolvent(
    forcefield,
    model="tip3p",
    padding=12.0 * angstrom,
)

omm_sol_topology = modeller.getTopology()
omm_sol_positions = modeller.getPositions()

omm_vac_system = forcefield.createSystem(
    omm_vac_topology,
    nonbondedMethod=NoCutoff,
    constraints=HBonds,
)


omm_sol_system = forcefield.createSystem(
    omm_sol_topology,
    nonbondedMethod=PME,
    nonbondedCutoff=9.0 * angstrom,
    rigidWater=True,
    constraints=HBonds,
)
omm_sol_system.addForce(openmm.MonteCarloBarostat(1 * atmosphere, temperature, 100))

### Save the system to xml files

In [ ]:
with open(saved_vacumm_system_file_name, "w") as f1:
    f1.write(openmm.XmlSerializer.serialize(omm_vac_system))

with open(saved_solvated_system_file_name, "w") as f2:
    f2.write(openmm.XmlSerializer.serialize(omm_sol_system))

### Save systems to PDB files

In [ ]:
with open(solvated_system_pdb_filename, "w") as f1:
    PDBFile.writeFile(omm_sol_topology, omm_sol_positions, f1)

with open(system_pdb_filename, "w") as f2:
    PDBFile.writeFile(omm_vac_topology, omm_vac_position, f2)